# Multiprocessing
## The only way I know how to do it, sorry

Multiprocessing can be useful and very easy to implement in Python, when you have a problem which needs a lot of calls to a certain function and they results of those calls are independent of each other, e.g. when you want to call another script that does something else entirely, but you have to call it many times and the execution time is long.
In general, the function you want to parallelize should take longer than a second otherwise it's probably not worth the overhead of distributing it to multiple cores. 

The most important thing to keep in mind is that parallelization requires whatever you WANT to parallelize to be a function call, i.e. you can only parallelize function calls, not code snippets. If what you want to parallelize is not already a function you need to make one. Let's do this!

In [132]:
import multiprocessing

In [133]:
dir(multiprocessing)

['Array',
 'AuthenticationError',
 'Barrier',
 'BoundedSemaphore',
 'BufferTooShort',
 'Condition',
 'Event',
 'JoinableQueue',
 'Lock',
 'Manager',
 'Pipe',
 'Pool',
 'Process',
 'ProcessError',
 'Queue',
 'RLock',
 'RawArray',
 'RawValue',
 'SUBDEBUG',
 'SUBWARNING',
 'Semaphore',
 'SimpleQueue',
 'TimeoutError',
 'Value',
 '__all__',
 '__builtins__',
 '__cached__',
 '__doc__',
 '__file__',
 '__loader__',
 '__name__',
 '__package__',
 '__path__',
 '__spec__',
 'active_children',
 'allow_connection_pickling',
 'connection',
 'context',
 'cpu_count',
 'current_process',
 'freeze_support',
 'get_all_start_methods',
 'get_context',
 'get_logger',
 'get_start_method',
 'log_to_stderr',
 'pool',
 'popen_fork',
 'process',
 'queues',
 'reduction',
 'set_executable',
 'set_forkserver_preload',
 'set_start_method',
 'synchronize',
 'sys',
 'util']

In [134]:
ncpu = multiprocessing.cpu_count()

In [135]:
from multiprocessing import Pool

In [136]:
# Function you want to parallelize
def square_input(num):
    return num*num

In [137]:
parameter_list = [1,2,3,4,5]

In [138]:
pool1 = Pool(processes=2)

In [139]:
result = pool1.map(square_input, parameter_list)

Map is a function that applies another function to an iterable (e.g., list, array) and returns the list of results
In multiprocessing this is one of the easiest way to parallelize. The pool that will keep all our task has this function as well.

It will break down the parameter list in chunks of the number of processes you specified in the initialization of the pool object, and process this many function evaluations in parallel.

The good thing about all the below examples is that they preserve the order of the output with respect to the input parameter list, regardless of the time a single evaluation takes.

In [140]:
result

[1, 4, 9, 16, 25]

The problem with map is that it maps exactly one parameter into a function, but usually the functions we want to parallelize have more than one input parameter.

Let's start a new example for this.

In [141]:
pool2 = Pool(processes=2)

In [142]:
# A function with multiple inputs
def add_numbers(num1, num2):
    return num1 + num2

# List of parameter combinations you want to evaluate the function with.
parameters_for_evaluation = [[1,1], [2,2], [6,5], [2,4]]

This requires the map-function to unpack the argument list and assign each parameter in the list to an input parameter in the add-numbers function. In Python 3 there is a convenient variation of map, called starmap, which does this for us.
(The star is a reference to the *-operator that can be used to unpack arguments)

In [143]:
result2 = pool2.starmap(add_numbers, parameters_for_evaluation)

In [144]:
result2

[2, 4, 11, 6]

One drawback of the normal pool.map / starmap, is that is works on N processes at the same time (here two), and waits until all of those processes are finished before it starts the next batch of N processes. If the execution of the function can vary a lot depending on the parameters, this is very inefficient. One should use map_async or starmap_async in this case. One difference is that after calling map_async, the rest of the program continues to run and you have to manually retrieve the results later. However, we can get the same behavior as the normal pool.map function when we add call the function wait() of the pool object. This way the rest of the program will only continue after all the whole pool of tasks has been finished. Afterwords we can retrieve the results with the get() method.

In [145]:
pool3 = Pool(processes=2)
result3 = pool3.starmap_async(add_numbers, parameters_for_evaluation)
result3.wait()

In [146]:
result3.get()

[2, 4, 11, 6]

## Using functools to set standard parameters

If the function you want to parallelize has many input parameters, but only a handful of those are variable in your particular parallelization, it may be useful to define a new function which is a restricted version of the old.

    We can use this using functools.partial

In [147]:
from functools import partial

In [148]:
# slightly more complicated function...
def multiply_numbers(num1, num2, return_integer):
    if return_integer:
        return int(num1*num2)
    else:
        return num1*num2

Let's say for our parallelization we always want to return the values as integers, but don't want to make a parameter list which includes the same value (say True) as a third parameter. We can do the following.

In [149]:
integer_multiply_numbers = partial(multiply_numbers, return_integer=True)

Now we effectively have a new function called integer_multiply_numbers, which has only two input parameters, because return_integer has been fixed to True.

In [150]:
pool4 = Pool(processes=2)
result_new = pool4.starmap(integer_multiply_numbers, parameters_for_evaluation)

In [151]:
result_new

[1, 4, 30, 8]

## Happy parallelizing!

# itertools: One of the most undervalued standard package

On a related note, there is another way to, e.g. implement nested for-loops with the awesomeness that is itertools! But it can also do many things like, generate all permutations from an iterable object and many more. itertools is a package to work with iterables and always returns iterable objects. To display the results you either have to loop over it or put it in a list!

In [152]:
import itertools

In [153]:
numbers = [1,2,3,4]
for perm in itertools.permutations(numbers, 4):
    print(perm)

(1, 2, 3, 4)
(1, 2, 4, 3)
(1, 3, 2, 4)
(1, 3, 4, 2)
(1, 4, 2, 3)
(1, 4, 3, 2)
(2, 1, 3, 4)
(2, 1, 4, 3)
(2, 3, 1, 4)
(2, 3, 4, 1)
(2, 4, 1, 3)
(2, 4, 3, 1)
(3, 1, 2, 4)
(3, 1, 4, 2)
(3, 2, 1, 4)
(3, 2, 4, 1)
(3, 4, 1, 2)
(3, 4, 2, 1)
(4, 1, 2, 3)
(4, 1, 3, 2)
(4, 2, 1, 3)
(4, 2, 3, 1)
(4, 3, 1, 2)
(4, 3, 2, 1)


But now on to the original topic of arbitrarily nested for-loops

In [154]:
Teff = [500, 600, 700]
logg = [3,4,5,6]
feh = [-1, 0, 1]
radius = [1,2,3,4,5,6]
model_parameter = [Teff, logg, feh, radius]

In [155]:
model_parameter

[[500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6]]

The way you would naively loop over all the parameters that you have would be this:

In [156]:
%%time
for par1 in model_parameter[0]:
    for par2 in model_parameter[1]:
        for par3 in model_parameter[2]:
            for par4 in model_parameter[3]:
                print((Teff, logg, feh, radius))

([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 1], [1, 2, 3, 4, 5, 6])
([500, 600, 700], [3, 4, 5, 6], [-1, 0, 

In [157]:
%%time 
for parameter_combination in itertools.product(*model_parameter): 
    #the star operator above is actually the same unpacking operator as in the name starmap
    print(parameter_combination)

(500, 3, -1, 1)
(500, 3, -1, 2)
(500, 3, -1, 3)
(500, 3, -1, 4)
(500, 3, -1, 5)
(500, 3, -1, 6)
(500, 3, 0, 1)
(500, 3, 0, 2)
(500, 3, 0, 3)
(500, 3, 0, 4)
(500, 3, 0, 5)
(500, 3, 0, 6)
(500, 3, 1, 1)
(500, 3, 1, 2)
(500, 3, 1, 3)
(500, 3, 1, 4)
(500, 3, 1, 5)
(500, 3, 1, 6)
(500, 4, -1, 1)
(500, 4, -1, 2)
(500, 4, -1, 3)
(500, 4, -1, 4)
(500, 4, -1, 5)
(500, 4, -1, 6)
(500, 4, 0, 1)
(500, 4, 0, 2)
(500, 4, 0, 3)
(500, 4, 0, 4)
(500, 4, 0, 5)
(500, 4, 0, 6)
(500, 4, 1, 1)
(500, 4, 1, 2)
(500, 4, 1, 3)
(500, 4, 1, 4)
(500, 4, 1, 5)
(500, 4, 1, 6)
(500, 5, -1, 1)
(500, 5, -1, 2)
(500, 5, -1, 3)
(500, 5, -1, 4)
(500, 5, -1, 5)
(500, 5, -1, 6)
(500, 5, 0, 1)
(500, 5, 0, 2)
(500, 5, 0, 3)
(500, 5, 0, 4)
(500, 5, 0, 5)
(500, 5, 0, 6)
(500, 5, 1, 1)
(500, 5, 1, 2)
(500, 5, 1, 3)
(500, 5, 1, 4)
(500, 5, 1, 5)
(500, 5, 1, 6)
(500, 6, -1, 1)
(500, 6, -1, 2)
(500, 6, -1, 3)
(500, 6, -1, 4)
(500, 6, -1, 5)
(500, 6, -1, 6)
(500, 6, 0, 1)
(500, 6, 0, 2)
(500, 6, 0, 3)
(500, 6, 0, 4)
(500, 6, 0, 5)
(

In [158]:
list(itertools.product(*model_parameter)) 

[(500, 3, -1, 1),
 (500, 3, -1, 2),
 (500, 3, -1, 3),
 (500, 3, -1, 4),
 (500, 3, -1, 5),
 (500, 3, -1, 6),
 (500, 3, 0, 1),
 (500, 3, 0, 2),
 (500, 3, 0, 3),
 (500, 3, 0, 4),
 (500, 3, 0, 5),
 (500, 3, 0, 6),
 (500, 3, 1, 1),
 (500, 3, 1, 2),
 (500, 3, 1, 3),
 (500, 3, 1, 4),
 (500, 3, 1, 5),
 (500, 3, 1, 6),
 (500, 4, -1, 1),
 (500, 4, -1, 2),
 (500, 4, -1, 3),
 (500, 4, -1, 4),
 (500, 4, -1, 5),
 (500, 4, -1, 6),
 (500, 4, 0, 1),
 (500, 4, 0, 2),
 (500, 4, 0, 3),
 (500, 4, 0, 4),
 (500, 4, 0, 5),
 (500, 4, 0, 6),
 (500, 4, 1, 1),
 (500, 4, 1, 2),
 (500, 4, 1, 3),
 (500, 4, 1, 4),
 (500, 4, 1, 5),
 (500, 4, 1, 6),
 (500, 5, -1, 1),
 (500, 5, -1, 2),
 (500, 5, -1, 3),
 (500, 5, -1, 4),
 (500, 5, -1, 5),
 (500, 5, -1, 6),
 (500, 5, 0, 1),
 (500, 5, 0, 2),
 (500, 5, 0, 3),
 (500, 5, 0, 4),
 (500, 5, 0, 5),
 (500, 5, 0, 6),
 (500, 5, 1, 1),
 (500, 5, 1, 2),
 (500, 5, 1, 3),
 (500, 5, 1, 4),
 (500, 5, 1, 5),
 (500, 5, 1, 6),
 (500, 6, -1, 1),
 (500, 6, -1, 2),
 (500, 6, -1, 3),
 (500, 6, 

Now imagine you want to do something like enumerate with the nested for loop (i.e. you want the index adressing a multidimensional array). Since you are adressing a tuple enumerate will not work, but if you simple make another list containing number from 0 to to the length of individual parameters, you'll get what you want

In [159]:
model_index = [range(len(Teff)), range(len(logg)), range(len(feh)), range(len(radius))]

In [160]:
for index in itertools.product(*model_index):
    print(index)

(0, 0, 0, 0)
(0, 0, 0, 1)
(0, 0, 0, 2)
(0, 0, 0, 3)
(0, 0, 0, 4)
(0, 0, 0, 5)
(0, 0, 1, 0)
(0, 0, 1, 1)
(0, 0, 1, 2)
(0, 0, 1, 3)
(0, 0, 1, 4)
(0, 0, 1, 5)
(0, 0, 2, 0)
(0, 0, 2, 1)
(0, 0, 2, 2)
(0, 0, 2, 3)
(0, 0, 2, 4)
(0, 0, 2, 5)
(0, 1, 0, 0)
(0, 1, 0, 1)
(0, 1, 0, 2)
(0, 1, 0, 3)
(0, 1, 0, 4)
(0, 1, 0, 5)
(0, 1, 1, 0)
(0, 1, 1, 1)
(0, 1, 1, 2)
(0, 1, 1, 3)
(0, 1, 1, 4)
(0, 1, 1, 5)
(0, 1, 2, 0)
(0, 1, 2, 1)
(0, 1, 2, 2)
(0, 1, 2, 3)
(0, 1, 2, 4)
(0, 1, 2, 5)
(0, 2, 0, 0)
(0, 2, 0, 1)
(0, 2, 0, 2)
(0, 2, 0, 3)
(0, 2, 0, 4)
(0, 2, 0, 5)
(0, 2, 1, 0)
(0, 2, 1, 1)
(0, 2, 1, 2)
(0, 2, 1, 3)
(0, 2, 1, 4)
(0, 2, 1, 5)
(0, 2, 2, 0)
(0, 2, 2, 1)
(0, 2, 2, 2)
(0, 2, 2, 3)
(0, 2, 2, 4)
(0, 2, 2, 5)
(0, 3, 0, 0)
(0, 3, 0, 1)
(0, 3, 0, 2)
(0, 3, 0, 3)
(0, 3, 0, 4)
(0, 3, 0, 5)
(0, 3, 1, 0)
(0, 3, 1, 1)
(0, 3, 1, 2)
(0, 3, 1, 3)
(0, 3, 1, 4)
(0, 3, 1, 5)
(0, 3, 2, 0)
(0, 3, 2, 1)
(0, 3, 2, 2)
(0, 3, 2, 3)
(0, 3, 2, 4)
(0, 3, 2, 5)
(1, 0, 0, 0)
(1, 0, 0, 1)
(1, 0, 0, 2)
(1, 0, 0, 3)
(1, 0, 0, 4)

In [163]:
for index_combination, parameter_combination in zip(itertools.product(*model_index), itertools.product(*model_parameter)):
    print(index_combination)
    print(parameter_combination)

(0, 0, 0, 0)
(500, 3, -1, 1)
(0, 0, 0, 1)
(500, 3, -1, 2)
(0, 0, 0, 2)
(500, 3, -1, 3)
(0, 0, 0, 3)
(500, 3, -1, 4)
(0, 0, 0, 4)
(500, 3, -1, 5)
(0, 0, 0, 5)
(500, 3, -1, 6)
(0, 0, 1, 0)
(500, 3, 0, 1)
(0, 0, 1, 1)
(500, 3, 0, 2)
(0, 0, 1, 2)
(500, 3, 0, 3)
(0, 0, 1, 3)
(500, 3, 0, 4)
(0, 0, 1, 4)
(500, 3, 0, 5)
(0, 0, 1, 5)
(500, 3, 0, 6)
(0, 0, 2, 0)
(500, 3, 1, 1)
(0, 0, 2, 1)
(500, 3, 1, 2)
(0, 0, 2, 2)
(500, 3, 1, 3)
(0, 0, 2, 3)
(500, 3, 1, 4)
(0, 0, 2, 4)
(500, 3, 1, 5)
(0, 0, 2, 5)
(500, 3, 1, 6)
(0, 1, 0, 0)
(500, 4, -1, 1)
(0, 1, 0, 1)
(500, 4, -1, 2)
(0, 1, 0, 2)
(500, 4, -1, 3)
(0, 1, 0, 3)
(500, 4, -1, 4)
(0, 1, 0, 4)
(500, 4, -1, 5)
(0, 1, 0, 5)
(500, 4, -1, 6)
(0, 1, 1, 0)
(500, 4, 0, 1)
(0, 1, 1, 1)
(500, 4, 0, 2)
(0, 1, 1, 2)
(500, 4, 0, 3)
(0, 1, 1, 3)
(500, 4, 0, 4)
(0, 1, 1, 4)
(500, 4, 0, 5)
(0, 1, 1, 5)
(500, 4, 0, 6)
(0, 1, 2, 0)
(500, 4, 1, 1)
(0, 1, 2, 1)
(500, 4, 1, 2)
(0, 1, 2, 2)
(500, 4, 1, 3)
(0, 1, 2, 3)
(500, 4, 1, 4)
(0, 1, 2, 4)
(500, 4, 1, 5)
(0, 1, 2

Which corresponds to an N-dimensional enumerate! :)